In [147]:
from basketball_reference_web_scraper import client
from basketball_reference_web_scraper.data import Location, Outcome, PeriodType, Position, Team
import datetime 
import requests
from bs4 import BeautifulSoup

In [148]:
def team_mapping(team_name):
    team_name = team_name.upper()
    
    team_name_mapping = {
        "ATL": Team.ATLANTA_HAWKS,
        "BOS": Team.BOSTON_CELTICS,
        "BKN": Team.BROOKLYN_NETS,
        "CHA": Team.CHARLOTTE_HORNETS,
        "CHI": Team.CHICAGO_BULLS,
        "CLE": Team.CLEVELAND_CAVALIERS,
        "DAL": Team.DALLAS_MAVERICKS,
        "DEN": Team.DENVER_NUGGETS,
        "DET": Team.DETROIT_PISTONS,
        "GSW": Team.GOLDEN_STATE_WARRIORS,
        "HOU": Team.HOUSTON_ROCKETS,
        "IND": Team.INDIANA_PACERS,
        "LAC": Team.LOS_ANGELES_CLIPPERS,
        "LAL": Team.LOS_ANGELES_LAKERS,
        "MEM": Team.MEMPHIS_GRIZZLIES,
        "MIA": Team.MIAMI_HEAT,
        "MIL": Team.MILWAUKEE_BUCKS,
        "MIN": Team.MINNESOTA_TIMBERWOLVES,
        "NOP": Team.NEW_ORLEANS_PELICANS,
        "NYK": Team.NEW_YORK_KNICKS,
        "OKC": Team.OKLAHOMA_CITY_THUNDER,
        "ORL": Team.ORLANDO_MAGIC,
        "PHI": Team.PHILADELPHIA_76ERS,
        "PHX": Team.PHOENIX_SUNS,
        "POR": Team.PORTLAND_TRAIL_BLAZERS,
        "SAC": Team.SACRAMENTO_KINGS,
        "SAS": Team.SAN_ANTONIO_SPURS,
        "TOR": Team.TORONTO_RAPTORS,
        "UTA": Team.UTAH_JAZZ,
        "WAS": Team.WASHINGTON_WIZARDS,
    }
    
    return team_name_mapping[team_name]

def get_player_id(player_name):
    name_parts = player_name.split()
    last_name_initial = name_parts[-1][0].lower()
    formatted_name = '-'.join(name_parts).lower()
    
    search_url = f'https://www.basketball-reference.com/players/{last_name_initial}/'
    response = requests.get(search_url)
    
    if response.ok:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        for link in soup.select('th[data-stat="player"] a'):
            if link.text.lower() == player_name.lower():
                player_id = link['href'].split('/')[3].replace('.html', '')
                return player_id
    
    return None

In [ ]:
#Home/Away Game Stats
def avg_player_stats_away(player_id, stat_name, pp_line, location, opponent):
    cur_szn_yr = 2024
    game_list1 = client.regular_season_player_box_scores(player_identifier=player_id, season_end_year=cur_szn_yr)

    if location == "AWAY":
        games_locationfilter = [game for game in game_list1 if game['location'] == Location.AWAY]
    elif location == "HOME":
        games_locationfilter = [game for game in game_list1 if game['location'] == Location.HOME]


    above_threshold_games = [game for game in games_locationfilter if game[stat_name] > pp_line]
    percentage_above_threshold = (len(above_threshold_games) / len(games_locationfilter)) * 100 if len(games_locationfilter) > 0 else 0
    
    print(f"Percentage of away games where {player_id} scored more than {pp_line} {stat_name}: {percentage_above_threshold:.2f}%")

avg_player_stats_away('antetgi01', 'points_scored', 27.5, "HOME", "NYK" )



In [146]:
#vs Team 

def avg_player_stats_away(player_id, stat_name, pp_line, location, opponent):
    opponent_enum = team_mapping(opponent)

    cur_szn_yr = 2024
    game_list1 = client.regular_season_player_box_scores(player_identifier=player_id, season_end_year=cur_szn_yr)
    game_list2 = client.regular_season_player_box_scores(player_identifier=player_id, season_end_year=cur_szn_yr -1)

    gamelist_total = game_list1 + game_list2

    games_opponentfilter = [game for game in gamelist_total if game['opponent'] == opponent_enum]


    above_threshold_games = [game for game in games_opponentfilter if game[stat_name] > pp_line]
    percentage_above_threshold = (len(above_threshold_games) / len(games_opponentfilter)) * 100 if len(games_opponentfilter) > 0 else 0
    
    print(f"Percentage of games vs {opponent} where {player_id} scored more than {pp_line} {stat_name}: {percentage_above_threshold:.2f}%")

avg_player_stats_away('lavinza01', 'points_scored', 21, "HOME", "MIA")



Percentage of games vs MIA where lavinza01 scored more than 21 points_scored: 0.00%


In [156]:
#back N games stats

def avg_player_stats(player_id, stat_name, pp_line, num_games):
    cur_szn_yr = 2024
    
    aux_game_list1 = client.regular_season_player_box_scores(player_identifier=player_id, season_end_year=cur_szn_yr)
    aux_game_list1.reverse()
    aux_game_list2 = list()
    if(len(aux_game_list1) < num_games):
        aux_game_list2 = client.regular_season_player_box_scores(player_identifier=player_id, season_end_year=cur_szn_yr-1)
    aux_game_list2.reverse()
    game_list = []
    index = 0
    while index < len(aux_game_li:t1) and len(game_list) < num_games:
        game_list.append(aux_game_list1[index])
        index = index + 1
    index = 0
    while index < len(aux_game_list2) and len(game_list) < num_games:
        game_list.append(aux_game_list2[index])
        index = index + 1
    
    above_threshold_games = [game for game in game_list if game[stat_name] > pp_line]
    percentage_above_threshold = (len(above_threshold_games) / len(game_list)) * 100 if len(game_list) > 0 else 0

    print(f"Percentage of last {num_games} games where {player_id} scored more than {pp_line} {stat_name}: {percentage_above_threshold:.2f}%")
    

avg_player_stats('antetgi01', 'points_scored', 27.5, 5)
avg_player_stats('antetgi01', 'points_scored', 27.5, 10)
avg_player_stats('antetgi01', 'points_scored', 27.5, 15)

Percentage of last 5 games where antetgi01 scored more than 27.5 points_scored: 40.00%
Percentage of last 10 games where antetgi01 scored more than 27.5 points_scored: 50.00%
Percentage of last 15 games where antetgi01 scored more than 27.5 points_scored: 53.33%
